In [1]:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 149.9 MB 26 kB/s s eta 0:00:01
     |████████████████████████████████| 831.4 MB 1.1 kB/s  eta 0:00:017     |█████████████████████           | 548.2 MB 93.9 MB/s eta 0:00:04
  Attempting uninstall: torch
    Found existing installation: torch 1.7.1+cpu
    Uninstalling torch-1.7.1+cpu:
      Successfully uninstalled torch-1.7.1+cpu
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.8.2+cpu requires torch==1.7.1, but you have torch 1.9.0 which is incompatible.
torchtext 0.8.1 requires torch==1.7.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.7.2 requires torch==1.7.1, but you have torch 1.9.0 which is incompatible.
fastai 2.2.7 requires torch<1.8,>=1.7.0, but you have torch 1.9.0 which is incompatible.


In [2]:
import gc
import copy
import time
import random
import string
import os

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold

#Text Cleaning
from bs4 import BeautifulSoup
import re 

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AdamW

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")



2022-02-07 08:13:50.356688: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-02-07 08:13:50.357482: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# imports the torch_xla package
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.serialization as xser

'''import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils'''

'import torch_xla\nimport torch_xla.debug.metrics as met\nimport torch_xla.distributed.parallel_loader as pl\nimport torch_xla.utils.utils as xu\nimport torch_xla.core.xla_model as xm\nimport torch_xla.distributed.xla_multiprocessing as xmp\nimport torch_xla.test.test_utils as test_utils'

In [4]:
os.environ['XLA_USE_BF16']="1"
os.environ['XLA_TENSOR_ALLOCATOR_MAXSIZE'] = '100000000'

In [5]:
CONFIG = {"seed": 2021,
          "epochs": 10,
          "model_name": "unitary/unbiased-toxic-roberta",
          "train_batch_size": 8,
          "valid_batch_size": 16,
          "max_length": 512,
          "learning_rate": 1e-5,
          "epsilon" : 1e-6,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,
          "T_max": 500,
          "weight_decay": 1e-5,
          "n_fold": 1,
          "n_accumulate": 1,
          "num_classes": 1,
          "margin": 0.5,
          "patience": 4
          }

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/997 [00:00<?, ?B/s]

In [6]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [7]:
df_train = pd.read_csv('../input/train-jigsaw-rate/train.csv')
df_train.head()

,more_toxic,less_toxic
0,Banning guns and explosives doesn t make a sta...,Banning guns and explosives doesn t make a sta...
1,I m saying it hasn t been researched and yet t...,I m saying it hasn t been researched and yet t...
2,The fuck did I just read,what what the fuck did i just read
3,It s almost as if these leave voting money gru...,Leading Britain off a cliff and cheating their...
4,Bulgaria Croatia Czech Republic Denmark Hungar...,Bulgaria Croatia Czech Republic Denmark Hungar...


In [8]:
df_valid = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")
df_valid.head()

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


In [9]:
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text

In [10]:
'''df_train.less_toxic = df_train.less_toxic.apply(text_cleaning)
df_train.more_toxic = df_train.more_toxic.apply(text_cleaning)
df_valid.less_toxic = df_valid.less_toxic.apply(text_cleaning)
df_valid.more_toxic = df_valid.more_toxic.apply(text_cleaning)'''

'df_train.less_toxic = df_train.less_toxic.apply(text_cleaning)\ndf_train.more_toxic = df_train.more_toxic.apply(text_cleaning)\ndf_valid.less_toxic = df_valid.less_toxic.apply(text_cleaning)\ndf_valid.more_toxic = df_valid.more_toxic.apply(text_cleaning)'

In [11]:
df_train = df_train.sample(frac=1, random_state=CONFIG['seed'])
df_valid = df_valid.sample(frac=1, random_state=CONFIG['seed'])

In [12]:
class JigsawDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.more_toxic = df['more_toxic'].values
        self.less_toxic = df['less_toxic'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        more_toxic = self.more_toxic[index]
        less_toxic = self.less_toxic[index]
        inputs_more_toxic = self.tokenizer.encode_plus(
                                more_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        inputs_less_toxic = self.tokenizer.encode_plus(
                                less_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        target = 1
        
        more_toxic_ids = inputs_more_toxic['input_ids']
        more_toxic_mask = inputs_more_toxic['attention_mask']
        
        less_toxic_ids = inputs_less_toxic['input_ids']
        less_toxic_mask = inputs_less_toxic['attention_mask']
        
        
        return {
            'more_toxic_ids': torch.tensor(more_toxic_ids, dtype=torch.long),
            'more_toxic_mask': torch.tensor(more_toxic_mask, dtype=torch.long),
            'less_toxic_ids': torch.tensor(less_toxic_ids, dtype=torch.long),
            'less_toxic_mask': torch.tensor(less_toxic_mask, dtype=torch.long),
            'target': torch.tensor(target, dtype=torch.long)
        }


In [13]:
class JigsawModel(nn.Module):
    def __init__(self, model_name):
        super(JigsawModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768, CONFIG['num_classes'])
        
    def forward(self, ids, mask):
        out = self.model(input_ids=ids,attention_mask=mask,
                             output_hidden_states=False)
        out = self.drop(out[1])
        out = out[1]
        outputs = self.fc(out)
        
        return outputs

In [14]:
JigsawM = xmp.MpModelWrapper(JigsawModel(CONFIG['model_name']))

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at unitary/unbiased-toxic-roberta and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
def criterion(outputs1, outputs2, targets):
    return nn.MarginRankingLoss(margin=CONFIG['margin'])(outputs1, outputs2, targets)

In [16]:
def accuracy(y_more_toxic, y_less_toxic): 
    correct = torch.sum(torch.where(y_more_toxic>y_less_toxic, 1, 0))
    wrong = torch.sum(torch.where(y_more_toxic<=y_less_toxic, 1, 0))
    acc = correct / (correct + wrong)
    return acc.view(-1).cpu().detach().numpy()

In [17]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    score_batch = []

    tk0 = tqdm(dataloader, total=len(dataloader), desc="Training", disable=not xm.is_master_ordinal())
    start_time = time.time()
    
    for bi, d in enumerate(tk0):
            
        
        more_toxic_ids = d['more_toxic_ids'].to(device, dtype = torch.long)
        more_toxic_mask = d['more_toxic_mask'].to(device, dtype = torch.long)
        less_toxic_ids = d['less_toxic_ids'].to(device, dtype = torch.long)
        less_toxic_mask = d['less_toxic_mask'].to(device, dtype = torch.long)
        targets = d['target'].to(device, dtype=torch.long)

        batch_size = more_toxic_ids.size(0)

        more_toxic_outputs = model(more_toxic_ids, more_toxic_mask)
        less_toxic_outputs = model(less_toxic_ids, less_toxic_mask)
        loss = criterion(more_toxic_outputs, less_toxic_outputs, targets)
        loss = loss / CONFIG['n_accumulate']
            
        score_batch.append(accuracy(more_toxic_outputs, less_toxic_outputs))
        
        loss.backward()

        if (bi + 1) % CONFIG['n_accumulate'] == 0:

            xm.optimizer_step(optimizer)    
            #optimizer.step()
            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
        # since the loss is on all 8 cores, reduce the loss values and print the average
        loss_reduced = xm.mesh_reduce('loss_reduce',loss, lambda x: sum(x) / len(x)) 

        if bi % 100 == 0:
            xm.master_print(
                f"bi={bi}, {time.time()-start_time:<2.2f} - loss:{loss_reduced}"
            )
 
        running_loss += (loss_reduced.detach().item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size
        
        epoch_score = np.mean(score_batch)

        tk0.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                            LR=optimizer.param_groups[0]['lr'])

    del loss
    del running_loss
    del loss_reduced
    gc.collect()
    return epoch_loss, epoch_score


In [18]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    score_batch = []

    vk0 = tqdm(dataloader, total=len(dataloader), desc="Validation", disable=not xm.is_master_ordinal())
    
    for bi, d in enumerate(vk0):        
        more_toxic_ids = d['more_toxic_ids'].to(device, dtype = torch.long)
        more_toxic_mask = d['more_toxic_mask'].to(device, dtype = torch.long)
        less_toxic_ids = d['less_toxic_ids'].to(device, dtype = torch.long)
        less_toxic_mask = d['less_toxic_mask'].to(device, dtype = torch.long)
        targets = d['target'].to(device, dtype=torch.long)
        
        batch_size = more_toxic_ids.size(0)

        more_toxic_outputs = model(more_toxic_ids, more_toxic_mask)
        less_toxic_outputs = model(less_toxic_ids, less_toxic_mask)
        
        loss = criterion(more_toxic_outputs, less_toxic_outputs, targets)
        score_batch.append(accuracy(more_toxic_outputs, less_toxic_outputs))

        # since the loss is on all 8 cores, reduce the loss values and print the average
        loss_reduced = xm.mesh_reduce('loss_reduce',loss, lambda x: sum(x) / len(x)) 
        # master_print will only print once (not from all 8 cores)
        #xm.master_print(f'val. loss={loss_reduced}')
        
        running_loss += (loss_reduced.detach().item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        epoch_score = np.mean(score_batch)
        
        vk0.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss)   
    
    gc.collect()
    return epoch_loss, epoch_score


In [19]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [20]:
def run(rank, flags):
    torch.set_default_tensor_type('torch.FloatTensor')
    
    device = xm.xla_device() # our device (single TPU core)
    
    xm.set_rng_state(CONFIG['seed'], device)
    fold = FLAGS["fold"]

   
    
    
    train_dataset = JigsawDataset(df_train, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])
    valid_dataset = JigsawDataset(df_valid, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])

    
    # special sampler needed for distributed/multi-core (divides dataset among the replicas/cores/devices)
    train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset,
        num_replicas=xm.xrt_world_size(), #divide dataset among this many replicas
        rank=xm.get_ordinal(), #which replica/device/core
        shuffle=True)
    
    # define DataLoader with the defined sampler
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=CONFIG['train_batch_size'],
        sampler=train_sampler,
        num_workers=0,
        drop_last=True)
    
    # same as train but with valid data
    valid_sampler = torch.utils.data.distributed.DistributedSampler(
        valid_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=False)

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=CONFIG['valid_batch_size'],
        sampler=valid_sampler,
        num_workers=0,
        drop_last=False)
    
    
    train_loader = pl.MpDeviceLoader(train_loader, device) # puts the train data onto the current TPU core
    valid_loader = pl.MpDeviceLoader(valid_loader, device) # puts the valid data onto the current TPU core
    

    model = JigsawM.to(device) # put model onto the current TPU core

    lr = CONFIG['learning_rate'] * xm.xrt_world_size()
    optimizer = AdamW(model.parameters(), lr=lr, eps=CONFIG['epsilon'], weight_decay=CONFIG['weight_decay'])
    scheduler = fetch_scheduler(optimizer)

    gc.collect()

    best_epoch_loss = np.inf
    history = defaultdict(list)
    epochs_no_improve = 0

    best_model_wts = copy.deepcopy(model.state_dict())
    
    xm.master_print(f'========== training fold {FLAGS["fold"]} for {CONFIG["epochs"]} epochs ==========')
    for epoch in range(CONFIG["epochs"]):
        xm.master_print(f'EPOCH {i}:')
        # train one epoch
        train_epoch_loss, train_epoch_score = train_one_epoch(model, optimizer, scheduler,
                                                              train_loader, device, epoch)
                
        # validation one epoch
        val_epoch_loss, val_epoch_score = valid_one_epoch(model, valid_loader, device, epoch)

        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Train Score'].append(train_epoch_score)
        history['Valid Score'].append(val_epoch_score)
        
        xm.master_print(f"Train score {train_epoch_score}")
        xm.master_print(f"Valid score {val_epoch_score}")

        if val_epoch_loss <= best_epoch_loss:
            xm.master_print(f"Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            history["Best Loss"].append(best_epoch_loss)
            best_model_wts = copy.deepcopy(model.state_dict())
            xm.rendezvous('save_model')
    
            xm.master_print('save model')

            PATH =f'Loss-Fold-{FLAGS["fold"]}.bin'
    
            xm.save(model.state_dict(), PATH)
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= CONFIG["patience"]:
                xm.master_print('Early stopping!' )
                xm.master_print()
                break

        model.load_state_dict(best_model_wts)

        gc.collect()
    
    

In [21]:
for i in range(CONFIG['n_fold']):
    FLAGS={}
    FLAGS["fold"] = i
    start_time = time.time()
    xmp.spawn(run, args=(FLAGS,), nprocs=8, start_method='fork')
    print('time taken: ', time.time()-start_time)
    print('==============================================================================')



========== training fold 0 for 10 epochs ==========
EPOCH 0:


Training:   0%|          | 0/939 [00:00<?, ?it/s]

bi=0, 22.42 - loss:0.38427734375


Training:  11%|█         | 100/939 [03:49<11:18,  1.24it/s, Epoch=0, LR=7.25e-5, Train_Loss=0.429]

bi=100, 230.27 - loss:0.233154296875


...

bi=800, 635.91 - loss:0.182373046875


Training:  96%|█████████▌| 900/939 [11:54<00:31,  1.23it/s, Epoch=4, LR=1.85e-5, Train_Loss=0.322]

bi=900, 715.20 - loss:0.209228515625


Validation: 100%|██████████| 236/236 [01:32<00:00,  2.54it/s, Epoch=4, Valid_Loss=0.408]


Train score 0.7358892560005188
Valid score 0.7033898234367371
Early stopping!

time taken:  4436.113703012466
